In [49]:
# This notebook will guide you to make correct format of Huggingface dataset, in proper parquet format and visualizable in Huggingface dataset hub.
# We will take the example of the dataset "Otter-AI/MMVet" and convert it to the proper format.

In [50]:
from datasets import load_dataset

data_path = "/mnt/sfs-common/krhu/lmms-eval-internal/build_cvrr.py"
df = load_dataset(data_path, split="test").to_pandas()
df.head(10)

/mnt/sfs-common/krhu/miniconda3/envs/lmms_eval/lib/python3.9/site-packages/datasets/load.py:922: FutureWarning: The repository for build_cvrr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /mnt/sfs-common/krhu/lmms-eval-internal/build_cvrr.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


Generating test split: 152 examples [00:00, 9291.47 examples/s]

{'time_order_understanding': {'0': {'Q': 'Does the video depict a person performing an activity in the conventional direction along the track?', 'A': 'No, the person is running backwards, which challenges the common expectation of direction in track running activities.', 'DimensionName': 'Time order understanding', 'VideoID': '204.mp4'}, '1': {'Q': 'What is happening in the video?', 'A': 'The video features a person in a blue running outfit and sunglasses running backward on an athletics track. The footage is fast-forwarded, highlighting the continuous backward running, with a backdrop of a football stadium, hurdles, and tiered stands.', 'DimensionName': 'Time order understanding', 'VideoID': '204.mp4'}, '2': {'Q': "Is the person's gear more suitable for indoor activities or outdoor sports?", 'A': "The person's gear, consisting of a blue running kit and black sunglasses, is more suitable for outdoor activities, indicating the appropriateness of the attire for the context of an athletic

,Q,A,DimensionName,VideoID
0,Does the video depict a person performing an a...,"No, the person is running backwards, which cha...",Time order understanding,204.mp4
1,What is happening in the video?,The video features a person in a blue running ...,Time order understanding,204.mp4
2,Is the person's gear more suitable for indoor ...,"The person's gear, consisting of a blue runnin...",Time order understanding,204.mp4
3,Is the person running towards the camera or ru...,The person is running away from the camera by ...,Time order understanding,204.mp4
4,Is the person running in clockwise direction o...,The person is running in anticlockwise directi...,Time order understanding,204.mp4
5,Does the person drink anything in the video wh...,"No, the person is not shown to be drinking any...",Time order understanding,204.mp4
6,Is the video capturing an event happening indo...,"The video captures an outdoor event, suggested...",Time order understanding,204.mp4
7,What object is initially focused at the start ...,The video starts with a close-up of a small or...,Time order understanding,205.webm
8,Is the flower brought closer to or moved furth...,The flower is moved further away from the came...,Time order understanding,205.webm
9,Describe the motion of the hand holding the fl...,The hand holding the flower shows a continuous...,Time order understanding,205.webm


In [51]:
df.head(20)

,Q,A,DimensionName,VideoID
0,Does the video depict a person performing an a...,"No, the person is running backwards, which cha...",Time order understanding,204.mp4
1,What is happening in the video?,The video features a person in a blue running ...,Time order understanding,204.mp4
2,Is the person's gear more suitable for indoor ...,"The person's gear, consisting of a blue runnin...",Time order understanding,204.mp4
3,Is the person running towards the camera or ru...,The person is running away from the camera by ...,Time order understanding,204.mp4
4,Is the person running in clockwise direction o...,The person is running in anticlockwise directi...,Time order understanding,204.mp4
5,Does the person drink anything in the video wh...,"No, the person is not shown to be drinking any...",Time order understanding,204.mp4
6,Is the video capturing an event happening indo...,"The video captures an outdoor event, suggested...",Time order understanding,204.mp4
7,What object is initially focused at the start ...,The video starts with a close-up of a small or...,Time order understanding,205.webm
8,Is the flower brought closer to or moved furth...,The flower is moved further away from the came...,Time order understanding,205.webm
9,Describe the motion of the hand holding the fl...,The hand holding the flower shows a continuous...,Time order understanding,205.webm


In [52]:
from datasets import Dataset, Features, Value
import pandas as pd

# Define the features for the dataset
features = Features(
    {
        "Q": Value(dtype="string"),
        "A": Value(dtype="string"),
        "DimensionName": Value(dtype="string"),
        "VideoID": Value(dtype="string"),
    }
)

df_items = {
    "Q": [],
    "A": [],
    "DimensionName": [],
    "VideoID": [],
}

for idx, row in df.iterrows():
    df_items["Q"].append(str(row["Q"]))
    df_items["A"].append(str(row["A"]))
    df_items["DimensionName"].append(str(row["DimensionName"]))
    df_items["VideoID"].append(str(row["VideoID"]))

df_items = pd.DataFrame(df_items)

In [53]:
df_items.head()

,Q,A,DimensionName,VideoID
0,Does the video depict a person performing an a...,"No, the person is running backwards, which cha...",Time order understanding,204.mp4
1,What is happening in the video?,The video features a person in a blue running ...,Time order understanding,204.mp4
2,Is the person's gear more suitable for indoor ...,"The person's gear, consisting of a blue runnin...",Time order understanding,204.mp4
3,Is the person running towards the camera or ru...,The person is running away from the camera by ...,Time order understanding,204.mp4
4,Is the person running in clockwise direction o...,The person is running in anticlockwise directi...,Time order understanding,204.mp4


In [54]:
dataset = Dataset.from_pandas(df_items, features=features)

In [55]:
hub_dataset_path = "lmms-lab/CVRR-ES"
dataset.push_to_hub(repo_id=hub_dataset_path, split="test", config_name="unusual_and_physically_anomalous_activities")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/CVRR-ES/commit/dd4d8ad2c74226ee6d3d8d53b2bda853e6e0503f', commit_message='Upload dataset', commit_description='', oid='dd4d8ad2c74226ee6d3d8d53b2bda853e6e0503f', pr_url=None, pr_revision=None, pr_num=None)